## Description

Demonstration on how the outliers are being removed from the database on a single parameter DF. The same principle is followed on the OutliersRemovalTools class. The only difference is that on the class, the methods updates the preprocessed_df attribute. 

Original docstrings from the class:

        '''
        This is a modified version of the remove_std_outliers_v1 method that deletes outliers that fits the following statements:

        There must be at least 2 consecutive values higher or lower than the 3 std dev scalar.
        from those 2 consecutive values, one must have a negative value and the other a positive value in order to be removed.

        :return: Updates the preprocessed_df attribute which then can be exported to a .csv file.
        '''


## Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import os



## Reading the .csv preprocessed files

In [2]:
#defining paths
preprocessed_path = r"C:\Users\victo\PycharmProjects\DataScienceProj\DS-Proj\Air_modelling\data\preprocessed_data\Parameters"
os.chdir(preprocessed_path)
preprocessed_fileslist = os.listdir()
#calling the fifth .csv file to work on 
#in this case, it will be PM10 data

#select the file or files to apply 

raw_P_df = pd.read_csv(preprocessed_fileslist[4])
raw_P_df.head()

,FECHAHORA,ATM,OBL,LPIN,SFE,TLA,VAL,CEN,AGU,LDO,MIR,FECHA,HORA
0,2016-01-01 00:00:00,146.95,197.67,NaN,115.54,143.40,17.08,86.12,49.92,174.04,69.75,2016-01-01,00:00:00
1,2016-01-01 01:00:00,216.10,138.09,NaN,84.24,100.46,29.15,46.49,52.80,115.27,68.99,2016-01-01,01:00:00
2,2016-01-01 02:00:00,113.44,98.79,NaN,135.39,82.05,30.89,63.93,52.71,99.00,117.70,2016-01-01,02:00:00
3,2016-01-01 03:00:00,73.30,97.94,NaN,117.60,114.74,38.74,60.75,51.24,83.65,160.30,2016-01-01,03:00:00
4,2016-01-01 04:00:00,52.55,134.39,NaN,164.68,118.83,51.48,108.09,58.84,49.70,180.89,2016-01-01,04:00:00


In [3]:
#eliminate columns we don't need for the moment such as FECHA and HORA
raw_P_df.columns.values
P_df = raw_P_df[['AGU', 'ATM', 'CEN', 'LDO', 'LPIN', 'MIR', 'OBL', 'SFE', 'TLA', 'VAL']]
P_df.head()

,AGU,ATM,CEN,LDO,LPIN,MIR,OBL,SFE,TLA,VAL
0,49.92,146.95,86.12,174.04,NaN,69.75,197.67,115.54,143.40,17.08
1,52.80,216.10,46.49,115.27,NaN,68.99,138.09,84.24,100.46,29.15
2,52.71,113.44,63.93,99.00,NaN,117.70,98.79,135.39,82.05,30.89
3,51.24,73.30,60.75,83.65,NaN,160.30,97.94,117.60,114.74,38.74
4,58.84,52.55,108.09,49.70,NaN,180.89,134.39,164.68,118.83,51.48


In [4]:
#Convert P_df into ndarray
P_arr = P_df.to_numpy()

#Create fvout_arr (first value out array) which has all the values but the first one
fvout_arr = P_arr[1:,:]

#Create a lvout_arr (last value out array) which has all the values but the last one 
lvout_arr = P_arr[:-1,:]


In [5]:
#create a delta_arr array that stores the value of the diff between fvout and lvout
delta_arr = fvout_arr - lvout_arr

#obtain the mean and std of the delta_arr values by station
mean_delta_arr = np.nanmean(delta_arr, axis=0)
std_delta_arr = np.nanstd(delta_arr, axis=0)

#create a std_factor var to specify the span of the scalar size
std_factor = 3 

#hscalar represents the highest value our parameter can have before we remove it 
#lscalar works the same but with the lowest value
hscalar = mean_delta_arr + std_factor * std_delta_arr 
lscalar = mean_delta_arr - std_factor * std_delta_arr 



In [6]:

#get the index of the elements whose values are gt hscalar or lt lscalar
#IMPORTANT!!: add a + 1 on the row index as we are going to delete the values from the main ndarray and not from delta_arr

outliers = np.where((delta_arr <= lscalar) | (delta_arr >= hscalar))
coordinates = list(zip(outliers[0], outliers[1]))


To consider: The delta_arr matrix has 1 row less than the original P_arr matrix, because of that, we must add 1 unit to the coordinates of the outliers obtained on the previous selection to access the original outlier coming from P_arr. 

In [7]:
#Create a new list to store the real outliers
coord_v2 = list()

for i in range(len(coordinates)):

    #check the next value from the first outliers list comparing it with the lscalar

    if delta_arr[(coordinates[i][0]+1, coordinates[i][1])] <= lscalar[coordinates[i][1]]:

        #check for differences on the signs between values, if they differ, add the loc to the new list
        if np.sign(delta_arr[(coordinates[i][0], coordinates[i][1])]) != np.sign(delta_arr[(coordinates[i][0]+1, coordinates[i][1])]):

            coord_v2.append((coordinates[i][0]+1, coordinates[i][1]))

    #check the next value from the first outliers list comparing it with the hscalar
    elif delta_arr[(coordinates[i][0]+1, coordinates[i][1])] >= hscalar[coordinates[i][1]]:

        #check for differences on the signs between values, if they differ, add the loc to the new list
        if np.sign(delta_arr[(coordinates[i][0], coordinates[i][1])]) != np.sign(delta_arr[(coordinates[i][0]+1, coordinates[i][1])]):

            coord_v2.append((coordinates[i][0]+1, coordinates[i][1]))
    else:
        
        continue

In [20]:
#Total qty of data in the matrix
t_datos = P_arr.shape[0] * P_arr.shape[1]
print('Total fields in the matrix: {}'.format(t_datos))

Total fields in the matrix: 350640


In [21]:
#Total qty of nan in the matrix
t_nan = np.isnan(P_arr).sum()
print('Total qty of nan in the matrix: {}'.format(t_nan))
print('Percentage of NaN for this matrix: {0:.2f}%'.format((t_nan/t_datos)*100))

Total qty of nan in the matrix: 117859
Percentage of NaN for this matrix: 33.61%


In [23]:
#total qty of 
t_real = t_datos - t_nan
print('Total qty of data in the matrix: {}'.format(t_real))

Total qty of data in the matrix: 232781


In [29]:
#data removed

data_to_remove = len(coord_v2)/(t_real)
print('Qty of data removed: {} or {:.2f}% from the total of real data.'.format(len(coord_v2), data_to_remove*100))


Qty of data removed: 568 or 0.24% from the total of real data.


In [31]:
#Elimination of outliers
for i in range(len(coord_v2)):
    P_arr[coord_v2[i]] = np.nan

In [32]:
#New df with outliers removed
processed_P_df = pd.DataFrame(columns=P_df.columns.values, data=P_arr)
processed_P_df.head()

,AGU,ATM,CEN,LDO,LPIN,MIR,OBL,SFE,TLA,VAL
0,49.92,146.95,86.12,174.04,NaN,69.75,197.67,115.54,143.40,17.08
1,52.80,NaN,46.49,115.27,NaN,68.99,138.09,84.24,100.46,29.15
2,52.71,113.44,63.93,99.00,NaN,117.70,98.79,135.39,82.05,30.89
3,51.24,73.30,60.75,83.65,NaN,160.30,97.94,117.60,114.74,38.74
4,58.84,52.55,108.09,49.70,NaN,180.89,134.39,164.68,118.83,51.48


In [33]:
# add date and time values
processed_P_df['FECHA'] = raw_P_df['FECHA']
processed_P_df['HORA'] = raw_P_df['HORA']
processed_P_df['FECHAHORA'] = raw_P_df['FECHAHORA']
processed_P_df.set_index('FECHAHORA')

,AGU,ATM,CEN,LDO,LPIN,MIR,OBL,SFE,TLA,VAL,FECHA,HORA
FECHAHORA,,,,,,,,,,,,
2016-01-01 00:00:00,49.92,146.95,86.12,174.04,NaN,69.75,197.67,115.540,143.40,17.08,2016-01-01,00:00:00
2016-01-01 01:00:00,52.80,NaN,46.49,115.27,NaN,68.99,138.09,84.240,100.46,29.15,2016-01-01,01:00:00
2016-01-01 02:00:00,52.71,113.44,63.93,99.00,NaN,117.70,98.79,135.390,82.05,30.89,2016-01-01,02:00:00
2016-01-01 03:00:00,51.24,73.30,60.75,83.65,NaN,160.30,97.94,117.600,114.74,38.74,2016-01-01,03:00:00
2016-01-01 04:00:00,58.84,52.55,108.09,49.70,NaN,180.89,134.39,164.680,118.83,51.48,2016-01-01,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,NaN,18.10,22.27,84.00,46.2,NaN,NaN,55.101,12.68,16.20,2019-12-31,19:00:00
2019-12-31 20:00:00,NaN,NaN,27.51,84.40,57.4,NaN,NaN,114.590,50.31,14.00,2019-12-31,20:00:00
2019-12-31 21:00:00,NaN,NaN,28.60,75.30,151.5,NaN,NaN,134.160,6.86,22.90,2019-12-31,21:00:00
